<a href="https://colab.research.google.com/github/yecatstevir/teambrainiac/blob/main/source/group_svm/Statistical_tests_Beta_maps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Two Sample T-test of Beta Maps between Young Adults and Adolescents 
Author: Stacey Rivet Beck

- Whole Brain Analyses between groups on bmaps
- Bmap Analyses within groups using bmaps of the ROI from the whole brain SVM training and the bmap from ROI data trained on SVM 
- Between Groups using bmaps of the ROI from the whole brain SVM training
- Between Groups using the bmap from ROI data trained on SVM

###Steps in this notebook: 
1) Connect to Google Drive, clone repository and load config file to access data on AWS

2) Import libraries

5) Load locally stored pickle files of each ROI containing the beta maps and assign to variables for each group for all ROIs

6) Load Whole brain (WB) beta maps stored on AWS and assign to variables 

7) Check summary statistics between groups for whole brain

8) Run stat tests on WB

8) For each Region, create the bmap using the whole brain data.

10) Check summary statistics 

9) Perform Two Tailed T-tests for between groups and within groups

10) Plot in a chart the significant results

In [ ]:
from google.colab import drive, files
drive.mount('/content/gdrive')#, force_remount = True)

# Clone the entire repo.
!git clone -l -s https://github.com/yecatstevir/teambrainiac.git
# Change directory into cloned repo
%cd teambrainiac/source/group_svm
!ls
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
# Import libraries
!pip install boto3 nilearn
from process import *
from access_data import *
from scipy import stats
import pandas as pd
import numpy as np

> Load ROI Bmaps

In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict
adpfc = open_pickle('/content/gdrive/MyDrive/data/AD_detrend_2_3_mPFC.pkl')
adnacc = open_pickle('/content/gdrive/MyDrive/data/AD_detrend_2_3_nacc_aal.pkl')
adacc = open_pickle('/content/gdrive/MyDrive/data/AD_detrend_2_3_acc_aal.pkl')
adai = open_pickle('/content/gdrive/MyDrive/data/AD_detrend_2_3_anterior_insula_aal.pkl')

as_mpfc = adpfc['bmap2'][0]
as_nacc = adnacc['bmap2'][0]
as_acc = adacc['bmap2'][0]
as_ai = adai['bmap2'][0]

yapfc = open_pickle('/content/gdrive/MyDrive/data/YA_detrend_2_3_mPFC.pkl')
yanacc = open_pickle('/content/gdrive/MyDrive/data/YA_detrend_2_3_nacc_aal.pkl')
yaacc = open_pickle('/content/gdrive/MyDrive/data/YA_detrend_2_3_acc_aal.pkl')
yaai = open_pickle('/content/gdrive/MyDrive/data/YA_detrend_2_3_anterior_insula_aal.pkl')

ya_mpfc = yapfc['bmap2'][0]
ya_nacc = yanacc['bmap2'][0]
ya_acc = yaacc['bmap2'][0]
ya_ai = yaai['bmap2'][0]

### Whole Brain Analysis Between Adolescent and Young Adult 2 Sample t-test

> Load Young Adult Whole Brain Information

In [ ]:
# Load model
#open path dictionary file to get subject ids
dict_path = "data/data_path_dictionary.pkl"
data_path_dict = open_pickle(dict_path)

# Load in affine matrix - helps map the data coordinates to real world coordinates
affine_image = access_load_data('w3rtprun_01.nii',False)
t1_image = access_load_data('MNI152.nii',False)

mask_type = 'mask' #'mask', 'masksubACC', 'masksubAI', 'masksubNAcc', 'masksubmPFC'
data_type = "YA_detrend"
runs_train = [1, 2]
runs_id = [i + 1 for i in runs_train]
m_path_ind = 0 #get sub_mask data in mask_data key
metric_ya = access_load_data(f'metrics/group_svm/{mask_type}/{data_type}_{runs_id}_{mask_type}_metrics.pkl', False)
print(metric_ya.keys())

dict_keys(['bmap3', 'bmap2_3', 'alphas1', 'bmap', 'y_train', 'test_preds', 'test_probs', 'test_acc', 'test_dfunc', 'y_t'])


> Load Adolescent Whole Brain Information

In [ ]:
# Load model
mask_type = 'mask' #'mask', 'masksubACC', 'masksubAI', 'masksubNAcc', 'masksubmPFC'
data_type = "AD_detrend"
runs_train = [1, 2]
runs_id = [i + 1 for i in runs_train]
m_path_ind = 0 #get sub_mask data in mask_data key
metric_ad = access_load_data(f'metrics/group_svm/{mask_type}/{data_type}_{runs_id}_{mask_type}_metrics.pkl', False)
print(metric_ad.keys())

dict_keys(['bmap3', 'bmap2_3', 'alphas1', 'bmap', 'y_train', 'test_preds', 'test_probs', 'test_acc', 'test_dfunc', 'y_t'])


> Whole Brain Summary Statistics

In [ ]:
Young_adult = metric_ya['bmap'][0][0]
Adolescent = metric_ad['bmap'][0][0]

dictionary_values = {'YA': Young_adult, 'AD':Adolescent}
summary_df = pd.DataFrame(dictionary_values)
summary_df.describe()

,YA,AD
count,237979.000000,237979.000000
mean,8.564183,0.321036
std,45.728512,45.985172
min,-203.502534,-306.102211
25%,-22.029241,-30.287586
50%,8.347217,0.310692
75%,38.758988,30.969864
max,237.847653,228.636860


In [ ]:
Young_adult = metric_ya['bmap'][0][0]
Adolescent = metric_ad['bmap'][0][0]

t_value, p_value = stats.ttest_ind(Young_adult, Adolescent)

print(f'Test statistic is {t_value}')

print(f'p-value for two tailed test is {p_value}')

alpha = 0.05

if p_value <= alpha:
  print(f'Conclusion: Since p-value {p_value} < alpha {alpha}, We reject the null hypothesis H0. \
  \nSo we conclude that at the whole brain level, Young Adults and Adolescents do not regulate the same. \
  \ni.e., μ1 = μ2 at {alpha} level of significance.')

else:

  print(f'Conclusion Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.')


Test statistic is 62.00711651640719
p-value for two tailed test is 0.0
Conclusion: Since p-value 0.0 < alpha 0.05, We reject the null hypothesis H0.   
So we conclude that at the whole brain level, Young Adults and Adolescents do not regulate the same.   
i.e., μ1 = μ2 at 0.05 level of significance.


###Medial Prefrontal Cortex Analysis Between Adolescent and Young Adult 2 Sample t-test

In [ ]:
# Load in the mask indices for mask overlay to create bmaps of a specific Region of interest
roi_path_ind = 1 # get mask data for region of interest
mask_roi = 'mPFC' #'acc_aal', 'anterior_insula_aal', 'mPFC', 'nacc_aal', 
indices_mask_roi = load_mask_indices(data_path_dict, mask_roi, roi_path_ind)

previous_bmap_ya = metric_ya['bmap2_3'][0]
bmap2ya = previous_bmap_ya.reshape(79 * 95 * 79)
bmap2_3ya = np.zeros((79, 95, 79))
bmap2_3ya = bmap2_3ya.reshape(79 * 95 * 79)
bmap2_3ya[indices_mask_roi] = bmap2ya[indices_mask_roi]

bmap2_3ya

mask shape: (79, 95, 79)


array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
# Load in the mask indices for mask overlay to create bmaps of a specific Region of interest
roi_path_ind = 1 # get mask data for region of interest
mask_roi = 'mPFC' #'acc_aal', 'anterior_insula_aal', 'mPFC', 'nacc_aal', 
indices_mask_roi = load_mask_indices(data_path_dict, mask_roi, roi_path_ind)

previous_bmap_ad = metric_ad['bmap2_3'][0]
bmap2ad = previous_bmap_ad.reshape(79 * 95 * 79)
bmap2_3ad = np.zeros((79, 95, 79))
bmap2_3ad = bmap2_3ad.reshape(79 * 95 * 79)
bmap2_3ad[indices_mask_roi] = bmap2ad[indices_mask_roi]

bmap2_3ad

mask shape: (79, 95, 79)


array([0., 0., 0., ..., 0., 0., 0.])

>> mPFC Sumamry Statistics

In [ ]:
dictionary_values = {'YA': bmap2_3ya, 'AD':bmap2_3ad}
summary_df = pd.DataFrame(dictionary_values)
summary_df.describe()

,YA,AD
count,592895.000000,592895.000000
mean,0.004181,0.011205
std,0.867464,0.980414
min,-112.903332,-61.773426
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,125.000130,136.057695


In [ ]:
t_value, p_value = stats.ttest_ind(bmap2_3ya, bmap2_3ad)

print(f'Test statistic is {t_value}')

print(f'p-value for two tailed test is {p_value}')

alpha = 0.05

if p_value <= alpha:
  print(f'Conclusion: Since p-value {p_value} < alpha {alpha}, We reject the null hypothesis H0. \
  \nSo we conclude that Young Adults and Adolescents may not regulate the same in the medial Prefrontal Cortex. \
  \ni.e., μ1 = μ2 at {alpha} level of significance.')

else:

  print(f'Conclusion: Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.\
  \n and therefore the Medial Prefrontal Cortex may play an equal role in regulation for both Young Adults and \
  \n Adolescents')


Test statistic is -4.131700751771177
p-value for two tailed test is 3.6011367639982006e-05
Conclusion: Since p-value 3.6011367639982006e-05 < alpha 0.05, We reject the null hypothesis H0.   
So we conclude that Young Adults and Adolescents may not regulate the same in the medial Prefrontal Cortex.   
i.e., μ1 = μ2 at 0.05 level of significance.


>> Compare ROI 

In [ ]:
dictionary_values = {'YA': ya_mpfc, 'AD':as_mpfc}
summary_df = pd.DataFrame(dictionary_values)
summary_df.describe()

,YA,AD
count,592895.000000,592895.000000
mean,0.001794,0.000812
std,0.570689,0.429639
min,-67.187558,-70.292464
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,83.962973,55.196536


In [ ]:
# Compare ROI extected from whole brain model to the actual ROI trained on ROI; also compare ROI AD to ROI YA

for groups in [[("Medial Prefrontal Cortex : Adolescent ROI on roi v Adolsecent ROI on WB for", as_mpfc, bmap2_3ad)], 
               [("Medial Prefrontal Cortex : Young Adult ROI on roi v ROI on WB", ya_mpfc, bmap2_3ya)], 
               [("Medial Prefrontal Cortex : Adolescent ROI v Young Adult ROI", as_mpfc, ya_mpfc)]]:
  
  for group, roi, roiwb in groups:

    t_value, p_value = stats.ttest_ind(roi, roiwb)

    print(f'Test statistic is {t_value}')

    print(f'p-value for two tailed test is {p_value}')

    alpha = 0.05

    if p_value <= alpha:
      print(f'{group} Conclusion: Since p-value  {p_value}  < alpha {alpha}, We reject the null hypothesis H0.')# \
      #\nSo we conclude Young Adults and Adolescents may not regulate the same at the Anterior Insula. \
      #\ni.e., μ1 = μ2 at {alpha} level of significance.')

    else:

      print(f'{group} Conclusion: Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.')
      #\n and therefore the whole brain model may not capture the significance of the Anterior Insula for both Young Adults and \
      #\n Adolescents')

Test statistic is -7.476116176697328
p-value for two tailed test is 7.660351253381644e-14
Medial Prefrontal Corext : Adolescent ROI on roi v Adolsecent ROI on WB for Conclusion: Since p-value  7.660351253381644e-14  < alpha 0.05, We reject the null hypothesis H0.
Test statistic is -1.7700375326755804
p-value for two tailed test is 0.07672114541939655
Medial Prefrontal Corext : Young Adult ROI on roi v ROI on WB Conclusion: Since p-value0.07672114541939655 > alpha 0.05,We do not reject the null hypothesis H0.
Test statistic is -1.0582085931566447
p-value for two tailed test is 0.28996057003888354
Medial Prefrontal Corext : Adolescent v Young Adult Conclusion: Since p-value0.28996057003888354 > alpha 0.05,We do not reject the null hypothesis H0.


###Nucleus Accumbens Analysis Between Adolescent and Young Adult 2 Sample t-test

In [ ]:
# Load in the mask indices for mask overlay to create bmaps of a specific Region of interest
roi_path_ind = 1 # get mask data for region of interest
mask_roi = 'nacc_aal' #'acc_aal', 'anterior_insula_aal', 'mPFC', 'nacc_aal', 
indices_mask_roi = load_mask_indices(data_path_dict, mask_roi, roi_path_ind)

previous_bmap_ya = metric_ya['bmap2_3'][0]
bmap2ya = previous_bmap_ya.reshape(79 * 95 * 79)
bmap2_3ya = np.zeros((79, 95, 79))
bmap2_3ya = bmap2_3ya.reshape(79 * 95 * 79)
bmap2_3ya[indices_mask_roi] = bmap2ya[indices_mask_roi]

bmap2_3ya

mask shape: (79, 95, 79)


array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
# Load in the mask indices for mask overlay to create bmaps of a specific Region of interest
roi_path_ind = 1 # get mask data for region of interest
mask_roi = 'nacc_aal' #'acc_aal', 'anterior_insula_aal', 'mPFC', 'nacc_aal', 
indices_mask_roi = load_mask_indices(data_path_dict, mask_roi, roi_path_ind)

previous_bmap_ad = metric_ad['bmap2_3'][0]
bmap2ad = previous_bmap_ad.reshape(79 * 95 * 79)
bmap2_3ad = np.zeros((79, 95, 79))
bmap2_3ad = bmap2_3ad.reshape(79 * 95 * 79)
bmap2_3ad[indices_mask_roi] = bmap2ad[indices_mask_roi]

bmap2_3ad

mask shape: (79, 95, 79)


array([0., 0., 0., ..., 0., 0., 0.])

>> Nucleus Accumbens Summary

In [ ]:
dictionary_values = {'YA': bmap2_3ya, 'AD':bmap2_3ad}
summary_df = pd.DataFrame(dictionary_values)
summary_df.describe()

,YA,AD
count,592895.000000,592895.000000
mean,0.007725,0.011626
std,1.379541,1.547360
min,-115.749862,-132.520353
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,123.001499,166.726276


In [ ]:
t_value, p_value = stats.ttest_ind(bmap2_3ya, bmap2_3ad)

print(f'Test statistic is {t_value}')

print(f'p-value for two tailed test is {p_value}')

alpha = 0.05

if p_value <= alpha:
  print(f'Conclusion: Since p-value  {p_value}  < alpha {alpha}, We reject the null hypothesis H0. \
  \nSo we conclude Young Adults and Adolescents may not regulate the same at the Nucleus Accumbens. \
  \ni.e., μ1 = μ2 at {alpha} level of significance.')

else:

  print(f'Conclusion: Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.\
  \n and therefore the Nucleus Accumbens may play an equal role in regulation for both Young Adults and \
  \n Adolescents')

Test statistic is -1.449044263538568
p-value for two tailed test is 0.14732548612940824
Conclusion: Since p-value0.14732548612940824 > alpha 0.05,We do not reject the null hypothesis H0.  
 and therefore the Nucleus Accumbens may play an equal role in regulation for both Young Adults and   
 Adolescents


>> Compare ROIs

In [ ]:
dictionary_values = {'YA': ya_nacc, 'AD':as_nacc}
summary_df = pd.DataFrame(dictionary_values)
summary_df.describe()

,YA,AD
count,592895.000000,592895.000000
mean,0.010081,0.000887
std,1.105725,0.826077
min,-111.846138,-63.049036
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,102.634047,64.208037


In [ ]:
# Compare ROI extected from whole brain model to the actual ROI trained on ROI; also compare ROI AD to ROI YA


for groups in [[("Nucleus Accumbens: Adolescent ROI on roi v Adolsecent ROI on WB", as_nacc, bmap2_3ad)], 
               [("Nucleus Accumbens: Young Adult ROI on roi v ROI on WB", ya_nacc, bmap2_3ya)], 
               [("Nucleus Accumbens: Adolescent v Young Adult", as_nacc, ya_nacc)]]:
  
  for group, roi, roiwb in groups:

    t_value, p_value = stats.ttest_ind(roi, roiwb)

    print(f'Test statistic is {t_value}')

    print(f'p-value for two tailed test is {p_value}')

    alpha = 0.05

    if p_value <= alpha:
      print(f'{group} Conclusion: Since p-value  {p_value}  < alpha {alpha}, We reject the null hypothesis H0.')# \
      #\nSo we conclude Young Adults and Adolescents may not regulate the same at the Anterior Insula. \
      #\ni.e., μ1 = μ2 at {alpha} level of significance.')

    else:

      print(f'{group} Conclusion: Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.')
      #\n and therefore the whole brain model may not capture the significance of the Anterior Insula for both Young Adults and \
      #\n Adolescents')

Test statistic is -4.7142927595035875
p-value for two tailed test is 2.4257921783996097e-06
Nucleus Accumbens: Adolescent ROI on roi v Adolsecent ROI on WB Conclusion: Since p-value  2.4257921783996097e-06  < alpha 0.05, We reject the null hypothesis H0.
Test statistic is 1.0262842648984607
p-value for two tailed test is 0.30475782134402185
Nucleus Accumbens: Young Adult ROI on roi v ROI on WB Conclusion: Since p-value0.30475782134402185 > alpha 0.05,We do not reject the null hypothesis H0.
Test statistic is -5.129363095217912
p-value for two tailed test is 2.9076972280392665e-07
Nucleus Accumbens: Adolescent v Young Adult Conclusion: Since p-value  2.9076972280392665e-07  < alpha 0.05, We reject the null hypothesis H0.


###Anterior Cingulate Cortex Analysis Between Adolescent and Young Adult 2 Sample t-test

In [ ]:
# Load in the mask indices for mask overlay to create bmaps of a specific Region of interest
roi_path_ind = 1 # get mask data for region of interest
mask_roi = 'acc_aal' #'acc_aal', 'anterior_insula_aal', 'mPFC', 'nacc_aal', 
indices_mask_roi = load_mask_indices(data_path_dict, mask_roi, roi_path_ind)

previous_bmap_ya = metric_ya['bmap2_3'][0]
bmap2ya = previous_bmap_ya.reshape(79 * 95 * 79)
bmap2_3ya = np.zeros((79, 95, 79))
bmap2_3ya = bmap2_3ya.reshape(79 * 95 * 79)
bmap2_3ya[indices_mask_roi] = bmap2ya[indices_mask_roi]

bmap2_3ya

mask shape: (79, 95, 79)


array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
# Load in the mask indices for mask overlay to create bmaps of a specific Region of interest
roi_path_ind = 1 # get mask data for region of interest
mask_roi = 'acc_aal' #'acc_aal', 'anterior_insula_aal', 'mPFC', 'nacc_aal', 
indices_mask_roi = load_mask_indices(data_path_dict, mask_roi, roi_path_ind)

previous_bmap_ad = metric_ad['bmap2_3'][0]
bmap2ad = previous_bmap_ad.reshape(79 * 95 * 79)
bmap2_3ad = np.zeros((79, 95, 79))
bmap2_3ad = bmap2_3ad.reshape(79 * 95 * 79)
bmap2_3ad[indices_mask_roi] = bmap2ad[indices_mask_roi]

bmap2_3ad

mask shape: (79, 95, 79)


array([0., 0., 0., ..., 0., 0., 0.])

>> Anterior Cingulate Cortex Summary

In [ ]:
dictionary_values = {'YA': bmap2_3ya, 'AD':bmap2_3ad}
summary_df = pd.DataFrame(dictionary_values)
summary_df.describe()

,YA,AD
count,592895.000000,592895.000000
mean,0.184045,0.068333
std,4.413037,3.392045
min,-129.489010,-131.767059
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,193.834653,175.091233


In [ ]:
t_value, p_value = stats.ttest_ind(bmap2_3ya, bmap2_3ad)

print(f'Test statistic is {t_value}')

print(f'p-value for two tailed test is {p_value}')

alpha = 0.05

if p_value <= alpha:
  print(f'Conclusion: Since p-value  {p_value}  < alpha {alpha}, We reject the null hypothesis H0. \
  \nSo we conclude Young Adults and Adolescents may not regulate the same at the Ant. Cingulate Cortex. \
  \ni.e., μ1 = μ2 at {alpha} level of significance.')

else:

  print(f'Conclusion: Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.\
  \n and therefore the Ant. Cingulate Cortex may play an equal role in regulation for both Young Adults and \
  \n Adolescents')

Test statistic is 16.00730451382578
p-value for two tailed test is 1.1522281988282325e-57
Conclusion: Since p-value  1.1522281988282325e-57  < alpha 0.05, We reject the null hypothesis H0.   
So we conclude Young Adults and Adolescents may not regulate the same at the Ant. Cingulate Cortex.   
i.e., μ1 = μ2 at 0.05 level of significance.


>> Compare ROI

In [ ]:
dictionary_values = {'YA': ya_acc, 'AD':as_acc}
summary_df = pd.DataFrame(dictionary_values)
summary_df.describe()

,YA,AD
count,592895.000000,592895.000000
mean,0.087415,0.019391
std,2.845394,1.922507
min,-114.438557,-111.246738
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,200.427386,86.794137


In [ ]:
# Compare ROI extected from whole brain model to the actual ROI trained on ROI; also compare ROI AD to ROI YA


for groups in [[("Ant. Cingulate Cortex: Adolescent ROI on roi v Adolsecent ROI on WB", as_acc, bmap2_3ad)], 
               [("Ant. Cingulate Cortex: Young Adult ROI on roi v ROI on WB", ya_acc, bmap2_3ya)], 
               [("Ant. Cingulate Cortex: Adolescent v Young Adult", as_acc, ya_acc)]]:
  
  for group, roi, roiwb in groups:

    t_value, p_value = stats.ttest_ind(roi, roiwb)

    print(f'Test statistic is {t_value}')

    print(f'p-value for two tailed test is {p_value}')

    alpha = 0.05

    if p_value <= alpha:
      print(f'{group} Conclusion: Since p-value  {p_value}  < alpha {alpha}, We reject the null hypothesis H0.')# \
      #\nSo we conclude Young Adults and Adolescents may not regulate the same at the Anterior Insula. \
      #\ni.e., μ1 = μ2 at {alpha} level of significance.')

    else:

      print(f'{group} Conclusion: Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.')
      #\n and therefore the whole brain model may not capture the significance of the Anterior Insula for both Young Adults and \
      #\n Adolescents')

Test statistic is -9.665537088107062
p-value for two tailed test is 4.231928995393069e-22
Ant. Cingulate Cortex: Adolescent ROI on roi v Adolsecent ROI on WB Conclusion: Since p-value  4.231928995393069e-22  < alpha 0.05, We reject the null hypothesis H0.
Test statistic is -14.170105060205996
p-value for two tailed test is 1.4149985848613655e-45
Ant. Cingulate Cortex: Young Adult ROI on roi v ROI on WB Conclusion: Since p-value  1.4149985848613655e-45  < alpha 0.05, We reject the null hypothesis H0.
Test statistic is -15.252899351931468
p-value for two tailed test is 1.5930483772663191e-52
Ant. Cingulate Cortex: Adolescent v Young Adult Conclusion: Since p-value  1.5930483772663191e-52  < alpha 0.05, We reject the null hypothesis H0.


###Anterior Insula (Right) Analysis Between Adolescent and Young Adult 2 Sample t-test

In [ ]:
# Load in the mask indices for mask overlay to create bmaps of a specific Region of interest
roi_path_ind = 1 # get mask data for region of interest
mask_roi = 'anterior_insula_aal' #'acc_aal', 'anterior_insula_aal', 'mPFC', 'nacc_aal', 
indices_mask_roi = load_mask_indices(data_path_dict, mask_roi, roi_path_ind)

previous_bmap_ya = metric_ya['bmap2_3'][0]
bmap2ya = previous_bmap_ya.reshape(79 * 95 * 79)
bmap2_3ya = np.zeros((79, 95, 79))
bmap2_3ya = bmap2_3ya.reshape(79 * 95 * 79)
bmap2_3ya[indices_mask_roi] = bmap2ya[indices_mask_roi]

bmap2_3ya

mask shape: (79, 95, 79)


array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
# Load in the mask indices for mask overlay to create bmaps of a specific Region of interest
roi_path_ind = 1 # get mask data for region of interest
mask_roi = 'anterior_insula_aal' #'acc_aal', 'anterior_insula_aal', 'mPFC', 'nacc_aal', 
indices_mask_roi = load_mask_indices(data_path_dict, mask_roi, roi_path_ind)

previous_bmap_ad = metric_ad['bmap2_3'][0]
bmap2ad = previous_bmap_ad.reshape(79 * 95 * 79)
bmap2_3ad = np.zeros((79, 95, 79))
bmap2_3ad = bmap2_3ad.reshape(79 * 95 * 79)
bmap2_3ad[indices_mask_roi] = bmap2ad[indices_mask_roi]

bmap2_3ad

mask shape: (79, 95, 79)


array([0., 0., 0., ..., 0., 0., 0.])

>> Anterior Insula Summary

In [ ]:
dictionary_values = {'YA': bmap2_3ya, 'AD':bmap2_3ad}
summary_df = pd.DataFrame(dictionary_values)
summary_df.describe()

,YA,AD
count,592895.000000,592895.000000
mean,0.002755,0.002354
std,0.826459,0.845693
min,-89.954536,-108.797203
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,105.864693,123.564931


In [ ]:
t_value, p_value = stats.ttest_ind(bmap2_3ya, bmap2_3ad)

print(f'Test statistic is {t_value}')

print(f'p-value for two tailed test is {p_value}')

alpha = 0.05

if p_value <= alpha:
  print(f'Conclusion: Since p-value  {p_value}  < alpha {alpha}, We reject the null hypothesis H0. \
  \nSo we conclude Young Adults and Adolescents may not regulate the same at the Anterior Insula. \
  \ni.e., μ1 = μ2 at {alpha} level of significance.')

else:

  print(f'Conclusion: Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.\
  \n and therefore the Anterior Insula may play an equal role in regulation for both Young Adults and \
  \n Adolescents')

Test statistic is 0.26110837009428334
p-value for two tailed test is 0.794008982469996
Conclusion: Since p-value0.794008982469996 > alpha 0.05,We do not reject the null hypothesis H0.  
 and therefore the Anterior Insula may play an equal role in regulation for both Young Adults and   
 Adolescents


>> Compare ROIs

In [ ]:
dictionary_values = {'YA': ya_ai, 'AD':as_ai}
summary_df = pd.DataFrame(dictionary_values)
summary_df.describe()

,YA,AD
count,592895.000000,592895.000000
mean,0.004822,0.000844
std,0.535734,0.370333
min,-56.906490,-54.966174
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,76.517413,47.493690


In [ ]:
# Compare ROI extected from whole brain model to the actual ROI trained on ROI; also compare ROI AD to ROI YA

for groups in [[("Anterior Insula: Adolescent ROI on roi v Adolsecent ROI on WB", as_ai, bmap2_3ad)], 
               [("Anterior Insula: Young Adult ROI on roi v ROI on WB", ya_ai, bmap2_3ya)], 
               [("Anterior Insula: Adolescent v Young Adult", as_ai, ya_ai)]]:
  
  for group, roi, roiwb in groups:

    t_value, p_value = stats.ttest_ind(roi, roiwb)

    print(f'Test statistic is {t_value}')

    print(f'p-value for two tailed test is {p_value}')

    alpha = 0.05

    if p_value <= alpha:
      print(f'{group} Conclusion: Since p-value  {p_value}  < alpha {alpha}, We reject the null hypothesis H0.')# \
      #\nSo we conclude Young Adults and Adolescents may not regulate the same at the Anterior Insula. \
      #\ni.e., μ1 = μ2 at {alpha} level of significance.')

    else:

      print(f'{group} Conclusion: Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.')
      #\n and therefore the whole brain model may not capture the significance of the Anterior Insula for both Young Adults and \
      #\n Adolescents')

Test statistic is -1.2594990036590081
p-value for two tailed test is 0.20785039778226308
Anterior Insula: Adolescent ROI on roi v Adolsecent ROI on WB Conclusion: Since p-value0.20785039778226308 > alpha 0.05,We do not reject the null hypothesis H0.
Test statistic is 1.6156799811654987
p-value for two tailed test is 0.1061637874118177
Anterior Insula: Young Adult ROI on roi v ROI on WB Conclusion: Since p-value0.1061637874118177 > alpha 0.05,We do not reject the null hypothesis H0.
Test statistic is -4.7028609836986455
p-value for two tailed test is 2.565699092117999e-06
Anterior Insula: Adolescent v Young Adult Conclusion: Since p-value  2.565699092117999e-06  < alpha 0.05, We reject the null hypothesis H0.


###Create Tables

In [ ]:
import pandas as pd
dictionary = {"y": {"Region": 'Whole Brain Beta Maps', "Test Statistic":62.00712, "p-value":0.0, "alpha":0.5, "Hypothesis": "Reject Null H0"},
              "yf": {"Region": "Medial Prefrontal Cortex from Whole Brain Beta Map", "Test Statistic":-4.13170, "p-value":'3.6011e-5', "alpha":0.5,"Hypothesis": "Reject Null H0"},
              "hgfj": {"Region":"Nucleus Accumbens from Whole Brain Beta Map","Test Statistic":-1.44904, "p-value": 0.14733, "alpha":0.5, "Hypothesis": "Fail to Reject Null H0"},
              "fdh": {"Region":"Anterior Cingulate Cortex from Whole Brain Beta Map","Test Statistic":16.00730, "p-value":'1.15223e-57', "alpha":0.5, "Hypothesis": "Reject Null H0"},
              "hfg": {"Region":"Anterior Insula (Right) from Whole Brain Beta Map","Test Statistic":0.26111, "p-value":0.79401, "alpha":0.5, "Hypothesis": "Fail to Reject Null H0"}
              }

df = pd.DataFrame(dictionary)
dff= df.T

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import plotly.io as pio

fig = go.Figure(data=[go.Table(
    header=dict(values=dff.columns,
                fill_color='#96c6e2',
                align='center'),
    cells=dict(values=[dff['Region'],dff['Test Statistic'], dff['p-value'], dff['alpha'],dff['Hypothesis']],
               fill_color='#eaeaf2',
               align='left'))
])

fig.update_layout(
    height=650,
    width = 800,
    title_text="Two Tailed T-Test Between Beta Maps for Young Adults and Adolescents",
    title_x=0.5
)

fig.show()

In [ ]:
import pandas as pd
dictionary = {
              "yf": {"Region": "Medial Prefrontal Cortex Beta map from Whole Brain versus Medial Prefrontal Cortex Beta Map ", "Test Statistic":-7.4762, "p-value":'7.660351e-14', "alpha":0.5,"Hypothesis": "Reject Null H0"},
              "hgfj": {"Region":"Nucleus Accumbens Beta map from Whole Brain versus Nucleus Accumbens Beta Map","Test Statistic":-4.714293, "p-value": '2.42579e-06', "alpha":0.5, "Hypothesis": "Reject Null H0"},
              "fdh": {"Region":"Anterior Cingulate Cortex Beta map from Whole Brain versus Anterior Cingulate Cortex Beta map","Test Statistic":-9.66554, "p-value":'4.23193e-22', "alpha":0.5, "Hypothesis": "Reject Null H0"},
              "hfg": {"Region":"Anterior Insula (Right) Beta map from Whole Brain versus Anterior Insula Beta map","Test Statistic":-1.25950, "p-value":0.20785, "alpha":0.5, "Hypothesis": "Fail to Reject Null H0"}
              }

df = pd.DataFrame(dictionary)
dff= df.T

import plotly.graph_objects as go
import pandas as pd
import plotly.io as pio

fig = go.Figure(data=[go.Table(
    header=dict(values=dff.columns,
                fill_color='#96c6e2',
                align='center'),
    cells=dict(values=[dff['Region'],dff['Test Statistic'], dff['p-value'], dff['alpha'],dff['Hypothesis']],
               fill_color='#eaeaf2',
               align='left'))
])

fig.update_layout(
    height=650,
    width = 900,
    title_text="Two Tailed T-Test Between ROI Beta Maps in Adolescents",
    title_x=0.5
)

fig.show()

In [ ]:
import pandas as pd
dictionary = {
              "yf": {"Region": "Medial Prefrontal Cortex Beta map from Whole Brain versus Medial Prefrontal Cortex Beta Map ", "Test Statistic":-1.77004, "p-value": 0.07672, "alpha":0.5,"Hypothesis": "Fail to Reject Null H0"},
              "hgfj": {"Region":"Nucleus Accumbens Beta map from Whole Brain versus Nucleus Accumbens Beta Map","Test Statistic":1.02629, "p-value": 0.30476, "alpha":0.5, "Hypothesis": "Fail to Reject Null H0"},
              "fdh": {"Region":"Anterior Cingulate Cortex Beta map from Whole Brain versus Anterior Cingulate Cortex Beta map","Test Statistic":-14.17011, "p-value":'1.414999e-45', "alpha":0.5, "Hypothesis": "Reject Null H0"},
              "hfg": {"Region":"Anterior Insula (Right) Beta map from Whole Brain versus Anterior Insula Beta map","Test Statistic":1.615678, "p-value":0.10616, "alpha":0.5, "Hypothesis": "Fail to Reject Null H0"}
              }

df = pd.DataFrame(dictionary)
dff= df.T

import plotly.graph_objects as go
import pandas as pd
import plotly.io as pio

fig = go.Figure(data=[go.Table(
    header=dict(values=dff.columns,
                fill_color='#96c6e2',
                align='center'),
    cells=dict(values=[dff['Region'],dff['Test Statistic'], dff['p-value'], dff['alpha'],dff['Hypothesis']],
               fill_color='#eaeaf2',
               align='left'))
])

fig.update_layout(
    height=650,
    width = 900,
    title_text="Two Tailed T-Test Between ROI Beta Maps in Young Adults",
    title_x=0.5
)

fig.show()

In [ ]:
import pandas as pd
dictionary = {
              "yf": {"Region": "Medial Prefrontal Cortex Beta map", "Test Statistic":-1.05821, "p-value": 0.28996, "alpha":0.5,"Hypothesis": "Fail to Reject Null H0"},
              "hgfj": {"Region":"Nucleus Accumbens Beta map","Test Statistic":-5.12936, "p-value": '2.90770e-07', "alpha":0.5, "Hypothesis": "Reject Null H0"},
              "fdh": {"Region":"Anterior Cingulate Cortex Beta map","Test Statistic":-15.25290, "p-value":'1.59305e-52', "alpha":0.5, "Hypothesis": "Reject Null H0"},
              "hfg": {"Region":"Anterior Insula (Right) Beta map","Test Statistic":-4.70286, "p-value":'2.5657e-06', "alpha":0.5, "Hypothesis": "Reject Null H0"}
              }

df = pd.DataFrame(dictionary)
dff= df.T

import plotly.graph_objects as go
import pandas as pd
import plotly.io as pio

fig = go.Figure(data=[go.Table(
    header=dict(values=dff.columns,
                fill_color='#96c6e2',
                align='center'),
    cells=dict(values=[dff['Region'],dff['Test Statistic'], dff['p-value'], dff['alpha'],dff['Hypothesis']],
               fill_color='#eaeaf2',
               align='left'))
])

fig.update_layout(
    height=650,
    width = 900,
    title_text="Two Tailed T-Test Between ROI Beta Maps for Young Adults and Adolescents",
    title_x=0.5
)

fig.show()

>Fin